In [1]:
import pandas as pd
import plotly.express as px
import plotly
from plotly.subplots import make_subplots
import numpy as np

In [2]:
df = pd.read_csv("cardio_train.csv", delimiter = ";")
df['age_in_years'] = round(df['age'] / 365.25)

BMI is calculated and categorized according to CDC (Center of Disease Control and Prevention).

In [3]:
df['BMI'] = round(df['weight']/((df['height']/100)**2), 1)

# Define conditions and corresponding categories
conditions = [
    (df['BMI'] <= 18.4),
    (df['BMI'] <= 24.9),
    (df['BMI'] <= 39.9),
    (df['BMI'] >= 40.0),
]

categories = [
    "Underweight",
    "Normal",
    "Overweight",
    "Obese"
    # Add more categories corresponding to your conditions
]

# Apply categorization using numpy.select
df['BMI'] = np.select(conditions, categories, default="Other")

In [4]:
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_in_years,BMI
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0,50.0,Normal
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1,55.0,Overweight
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1,52.0,Normal
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1,48.0,Overweight
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0,48.0,Normal


In [5]:
# Group the DataFrame and count the occurrences
counts = df.groupby(['age_in_years', 'cardio']).size().unstack(fill_value=0).reset_index()

# Reshape the data for Plotly
counts_melted = pd.melt(counts, id_vars='age_in_years', value_vars=[0, 1], var_name='Cardio', value_name='Count')

# Replace cardio values with descriptive labels
counts_melted['Cardio'] = counts_melted['Cardio'].map({0: 'No Disease', 1: 'Have Disease'})

# Plot the bar chart using Plotly
fig_age = px.bar(counts_melted, x='age_in_years', y='Count', color='Cardio', barmode='group',
             color_discrete_map={'No Disease': 'black', 'Have Disease': 'skyblue'})

# Update layout
fig_age.update_layout(title='Age of Patients with Cardio Vascular Disease',
                  xaxis_title='Age',
                  yaxis_title='Count of Patients',
                  xaxis=dict(range=[35, counts_melted['age_in_years'].max()]),
                  bargap=0.2, # Expand bar width
                  plot_bgcolor='white')

In [6]:
plotly.offline.plot(fig_age, filename="fig_age.html")

'fig_age.html'

In [7]:
# Group the DataFrame and count the occurrences
gender_count = df.groupby(['gender', 'cardio']).size().unstack(fill_value=0).reset_index()

# Reshape the data for Plotly
counts_melted = pd.melt(gender_count, id_vars='gender', value_vars=[0, 1], var_name='Cardio', value_name='Count')

# Replace cardio values with descriptive labels
counts_melted['Cardio'] = counts_melted['Cardio'].map({0: 'No Disease', 1: 'Have Disease'})
counts_melted['gender'] = counts_melted['gender'].map({1: 'Women', 2: 'Men'})

# Plot the bar chart using Plotly
fig_gender = px.bar(counts_melted, x='gender', y='Count', color='Cardio', barmode='group',
             color_discrete_map={'No Disease': 'black', 'Have Disease': 'skyblue'})

# Update layout
fig_gender.update_layout(title='Gender of Patients with Cardio Vascular Disease',
                  xaxis_title='Gender',
                  yaxis_title='Count of Patients',
                  bargap=0.2, # Expand bar width
                  plot_bgcolor='white')

In [8]:
plotly.offline.plot(fig_gender, filename="fig_gender.html")

'fig_gender.html'

In [9]:
# Group the DataFrame and count the occurrences
BMI_count = df.groupby(['BMI', 'cardio']).size().unstack(fill_value=0).reset_index()

# Reshape the data for Plotly
counts_melted = pd.melt(BMI_count, id_vars='BMI', value_vars=[0, 1], var_name='Cardio', value_name='Count')

# Replace cardio values with descriptive labels
counts_melted['Cardio'] = counts_melted['Cardio'].map({0: 'No Disease', 1: 'Have Disease'})

# Plot the bar chart using Plotly
fig_BMI = px.bar(counts_melted, x='BMI', y='Count', color='Cardio', barmode='group',
             color_discrete_map={'No Disease': 'black', 'Have Disease': 'skyblue'})

# Update layout
fig_BMI.update_layout(title='BMI of Patients with Cardio Vascular Disease',
                  xaxis_title='BMI',
                  yaxis_title='Count of Patients',
                  bargap=0.2, # Expand bar width
                  plot_bgcolor='white',
                  xaxis=dict(categoryorder='array', categoryarray=['Underweight', 'Normal', 'Overweight', 'Obese']))



In [10]:
plotly.offline.plot(fig_BMI, filename="fig_bmi.html")

'fig_bmi.html'

In [ ]:
# Show the plot
fig_age.show()
fig_gender.show()
fig_BMI.show()

In [ ]:
# Filter data for active individuals
active_data = counts_melted[counts_melted['active'] == 'Active']

# Plot the donut chart for active individuals
fig_active = px.pie(active_data, names='Cardio', values='Count', hole=0.5,
                    color='Cardio', color_discrete_map={'No Disease': 'black', 'Have Disease': 'skyblue'})
fig_active.update_layout(
    title='Active Patients with Cardiovascular Disease',
    plot_bgcolor='white'
)

# Filter data for inactive individuals
inactive_data = counts_melted[counts_melted['active'] == 'Not Active']

# Plot the donut chart for inactive individuals
fig_inactive = px.pie(inactive_data, names='Cardio', values='Count', hole=0.5,
                      color='Cardio', color_discrete_map={'No Disease': 'black', 'Have Disease': 'skyblue'})
fig_inactive.update_layout(
    title='Inactive Patients with Cardiovascular Disease',
    plot_bgcolor='white'
)

fig_active.show()
fig_inactive.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a dropdown menu to select gender
gender_dropdown = go.FigureWidget(
    data=[],
    layout=go.Layout(
        updatemenus=[
            dict(
                buttons=[
                    dict(label="All", method="update", args=[{"visible": True}, {"title": "All Patients"}]),
                    dict(label="Male", method="update", args=[{"visible": [True, False]}, {"title": "Male Patients"}]),
                    dict(label="Female", method="update", args=[{"visible": [False, True]}, {"title": "Female Patients"}])
                ],
                direction="down",
                showactive=True,
                x=0.1,
                xanchor="left",
                y=1.15,
                yanchor="top"
            )
        ]
    )
)

# Create the pie chart for active patients
fig_active_pie = go.Pie(labels=active_data['Cardio'], values=active_data['Count'],
                        marker=dict(colors=['black', 'skyblue']))

# Create the pie chart for inactive patients
fig_inactive_pie = go.Pie(labels=inactive_data['Cardio'], values=inactive_data['Count'],
                          marker=dict(colors=['black', 'skyblue']))

# Combine the pie charts
fig_combined = make_subplots(rows=1, cols=2, subplot_titles=("Active Patients", "Inactive Patients"))
fig_combined.add_trace(fig_active_pie, row=1, col=1)
fig_combined.add_trace(fig_inactive_pie, row=1, col=2)

# Add the dropdown menu to the layout
fig_combined.update_layout(
    title_text="Patients with Cardiovascular Disease",
    updatemenus=[gender_dropdown.layout.updatemenus[0]]
)

# Display the combined figure
fig_combined.show()


ValueError: Trace type 'pie' is not compatible with subplot type 'xy'
at grid position (1, 1)

See the docstring for the specs argument to plotly.subplots.make_subplots
for more information on subplot types